In [12]:
import numpy as np
from scipy.special import kv

def RasSoln(P, Q_max, r, s_ln, soln):
    s = np.exp(s_ln)
    omega = np.reshape(((2 * np.pi) / P), (r.size,1))
    
    # Fully confined analytical model from Rasmussen et al., (2003)
    if soln == 'confined':
        arg = np.sqrt((1j * r**2 * omega) / s[0])
    
    # Leaky analytical model from Rasmussen et al., (2003)
    elif soln == 'leaky':
        B_sq = s[1] / s[2]
        arg = np.sqrt(((1j * r**2 * omega) / s[0]) + (r**2 / B_sq))        

    else:
        print('Error: Pick a valid analytical model')

    # Complex valued phasor 
    phasor_mod = (Q_max / (2 * np.pi * s[1])) * kv(0, arg)

    # Fourier coefficient array used for inversion. Order is [real, imag] for each 
    y_mod = np.zeros([r.size*2,1])
    y_mod[0:-1:2] = phasor_mod.real
    y_mod[1::2] = phasor_mod.imag
    return y_mod
    
def jacobian(P, Q_max, r, s_ln, delta, soln):
    num_param = s_ln.size
    num_data = r.size*2
     
    J = np.zeros((num_data, num_param))
    coeffs_base = RasSoln(P, Q_max, r, s_ln, soln)
    
    for i in range(0, num_param):
        sj = s_ln
        sj[i] = s_ln[i] + delta[i]
        coeffs_mod = RasSoln(P, Q_max, r, sj, soln)
        J[:,i] = (coeffs_mod - coeffs_base) / delta[i]

    return(J)


In [13]:
P = np.array([[30], [90]])
Q_0 = np.array([[7e-5], [7e-5]])
r = np.array([[10], [10]])
s_true = np.array([[2], [-8]])
delta = np.array([[0.1], [0.1]])
soln = 'confined'

y_mod = RasSoln(P, Q_0, r, s_true, soln)
J = jacobian(P, Q_0, r, s_true, delta, soln)

ValueError: operands could not be broadcast together with shapes (4,1) (2,1) 

In [ ]:
print('y_mod = ', y_mod)

In [ ]:
print ('J = ', J)